# Script for comparing annotation
Currently, we are looking to compare the annotations between annotators regarding metaphors and non-metaphors. Note that all words/phrases that are tagged are only tagged with the label [METAPHOR]. We want a script that:
1. Compares words/phases that get tagged
2. Compares the tags associated with these words/phrases
3. Outputs some sort of agreement level between annotators

In the future, we also want some way to compare these annotations with the appraisal annotations for these articles to see what class of words tend to be classified as metaphors or non-metaphors.

In [1]:
import pandas as pd
from sklearn.metrics import cohen_kappa_score
from statistics import mean
from ast import literal_eval
import numpy as np

## Load annotations from LabelStudio output
LabelStudio can output the annotations in various formats, including json, csv, and tsv.

For simplicity we are using the csv option, which includes the following columns:
* **annotation_id** - unique id associated with each annotation
* **annotator** - id associated with a particular annotator
* **created_at** - time the annotation was created
* **id** - text id
* **label** - dictionary of labels associated with the text
  * "start": the index of the first labelled character within the selected text
  * "end": the index of the final labelled character within the selected text
  * "text": the text (word/phrase) that was selected to be labelled
  * "labels": a list of labels associated with the selected text
* **lead_time** - (not sure what this is)
* **text** - article text
* **updated_at** - time the annotation was last updated

Relevant columns here are annotator, id (rename to text_id), label, text

In [2]:
import os

In [3]:
os.getcwd()

'C:\\Users\\romha\\OneDrive - University of Waterloo\\Desktop\\Spr25\\metaphor_annotations'

In [4]:
input = 'mar4.csv'
annotations_df = pd.read_csv(input, usecols=["annotator", "id", "label", "text", "created_at", "updated_at"])
# rename 'id' to 'text_id' for clarification
annotations_df.rename(columns={"id": "text_id"}, inplace=True)

# reorder columns for clarification
order = ["text_id", "annotator", "text", "label"]
annotations_df = annotations_df.reindex(columns=order)

## Comparing tagged words and labels


Tentative idea for how to proceed:
* With just two annotators, we could manually split the annotations into two dataframes, and then merge them again on text id, giving us a new dataframe containing the text id, the article text, and two label columns from the annotators
* From there, we can write a function to compare the lists of dictionaries in the label columns, and then create the following additional columns in this new merged dataframe:
  * overlap_agreement - if there is an overlap between non-identical tags (e.g. "line" vs "line in the sand"), how much do they agree?
  * textual_agreement - of words/phrases tagged, how many are identical?

To filter by annotators, use the numbers found from the previous code block and replace them below.

In [19]:
# filter the dataframe to contain just copies of annotations from each annotator
annotator1_df = annotations_df.loc[(annotations_df['annotator'] == 4)]
annotator2_df = annotations_df.loc[(annotations_df['annotator'] == 5)]

Here we create a few helper functions to turn the label columns into useable data.

First, `labels_to_list` takes in the labels columns and returns a list of lists containing just the start and end indices of tagged words/phrases. These uniquely identify a tagged item within the text, and disambiguates between different instances of the same word (e.g. if "line" appears multiple times throughout the comment), so it is all the information we will need to work with.

Each item in the returned list is a list of [start index, end index pairs], extracted from the list of dictionaries of labels.

In [12]:
# helper function to build a list of lists containing the start and end indices
# and the difference between these indices
# where labels is the labels associated with a specific text
def labels_to_list(labels):
  annotations = []

  labels = literal_eval(labels)

  for label in labels:
    tags = []
    tags.append(int(label['start']))
    tags.append(int(label['end']))

    annotations.append(tags)

  return annotations

This is the main helper function that will produce textual and overlap agreement. As a reminder, textual agreement refers to the similarity in items that are tagged between the annotators (we are not counting the number of tags, but rather, the specific words that are tagged - it would not be very useful to know that they share the same number of annotations if they are both tagging different words). Overlap agreement looks specifically at items that may overlap, for example, if one annotator only tagged "line" as metaphor but the other tagged the entire phrase "line in the sand".

To determine the textual agreement, we first make the labels dictionaries into usable start-end index pairs, making sure to sort them in order (the csv output seems to list the labels in the order that they are created, not in the order the text appears in the comment). Then, we iterate through the two lists to create two additional lists - these contain the annotations that will later be used to compare against each other to produce cohen's kappa.

To produce the additional lists, we add in order the labels that appear in both lists; when a label only appears in one list, we append "0, 0" to the other list to denote that the item was not annotated by that annotator.

It is trickier to use cohen's kappa to determine overlap agreement. As such, I used a rudimentary/brute force method where for each overlaping (but, crucially, not identical) label, I find the ratio between the difference of the indices and add this to a third list for easier calculation.

As an example: let's say "line" has indices [0, 3] and "line in the sand" has indices [0, 15]. Taking the difference, "line" has a difference of `3` and "line in the sand" has a differnece of `15`. We then divide `3/15` to get `0.2`, telling us that the two tags only overlap by `20%`.

Once these ratios are calculated, we then find the mean of these overlap ratios to determine how closely the overlapping tags were to each other, with 0 being no overlap at all and 1 being 100% overlap (note: a 1 should never appear, as for comments where there is no overlap between annotations, `NaN` is instead appended to the row to avoid messing up final calculations).

Similarly, a 1 in textual agreement suggests 100% agreement, and a 0 indicates 0% agreement between the annotators.

In [13]:
# helper function to compare tags and labels for a single text
# where row contains the text and labels from both annotators associated with the text
# label1 and label2 are the column headers for the labels
# returns a list containing the agreement score
# of overlapping labels and overall tags
def compare_labels(row, label1, label2):

  # turn labels dictionaries into usable lists
  list1 = labels_to_list(row[label1])
  list2 = labels_to_list(row[label2])

  # sort the keys to be in index order
  # since LabelStudio orders it by time updated, not order of appearance in text
  list1 = sorted(list1, key=lambda x: x[0])
  list2 = sorted(list2, key=lambda x: x[0])

  tags1 = []
  tags2 = []
  overlap = []

  i = 0
  j = 0

  # cohen's kappa takes two lists of equal lengths to compare labels
  # so here we are building new lists from the annotations
  # appending 0's where the label exists in one list but not the other
  # and iterating through both lists until we have read both of them in full

  while i < len(list1) and j < len(list2):
    # if the current label in list1 appears before the current label in list2
    # append list1's label to tags1
    # and append the '0, 0' label to tags2
    if list1[i][0] < list2[j][0]:
      tags1.append(str(list1[i]))
      tags2.append("0, 0")
      i += 1

    # if the current label in list 1 starts with the same word instance
    # as the label in list2
    # append both labels to their respective lists
    elif list1[i][0] == list2[j][0]:
      tags1.append(str(list1[i]))
      tags2.append(str(list2[j]))

      # and here we check for overlap with the difference in index
      # e.g. if one label is [4, 10] and the other [4, 16]
      # we compare the difference of 6 and 12
      # and append 6/12, or 0.5, to the overlap list
      if list1[i][1] != list2[j][1]:
        diff1 = list1[i][1] - list1[i][0]
        diff2 = list2[j][1] - list2[j][0]
        numerator = min(diff1, diff2)
        denominator = max(diff1, diff2)
        overlap.append(float(numerator/denominator))

      i += 1
      j += 1

    # if the current label in list2 appears before the current label in list1
    # append list2's label to tags2
    # and append the '0, 0' label to tags1
    else:
      tags1.append("0, 0")
      tags2.append(str(list2[j]))
      j += 1

  # find cohen's kappa using our two tags arrays
  # if the tags arrays only contain 1 element (i.e. only has one label)
  # cohen's kappa returns a NaN
  # so the condition checks and corrects for that
  textual_kappa = cohen_kappa_score(tags1, tags2)
  if tags1 == tags2 and len(tags1) == 1:
    textual_kappa = 1

  # find the mean of the overlap agreement in the text
  # or append NaN if no overlap
  if overlap != []:
    overlap_agreement = mean(overlap)
  else:
    overlap_agreement = np.nan

  return [textual_kappa, overlap_agreement]

In [18]:
compare_df.head()

,text_id,text,label_1,label_2,textual_agreement,overlap_agreement
1,72,Nobody really cares if a woman or man holds th...,"[{""start"":322,""end"":326,""text"":""bill"",""labels""...","[{""start"":48,""end"":55,""text"":""highest"",""labels...",1.000000,NaN
2,131,"Funnily enough, if the Ontario Liberals tried ...","[{""start"":48,""end"":58,""text"":""shell game"",""lab...","[{""start"":48,""end"":58,""text"":""shell game"",""lab...",1.000000,NaN
3,146,Apple stuff consist of 95% marketing nonsense ...,"[{""start"":138,""end"":144,""text"":""thrive"",""label...","[{""start"":125,""end"":130,""text"":""knows"",""labels...",1.000000,NaN
4,178,This whole concept of 'national daycare' sound...,"[{""start"":72,""end"":76,""text"":""he##"",""labels"":[...","[{""start"":294,""end"":302,""text"":""draining"",""lab...",0.298246,NaN
5,185,'... a larger question is how to deal with Chi...,"[{""start"":139,""end"":143,""text"":""sold"",""labels""...","[{""start"":64,""end"":69,""text"":""petty"",""labels"":...",0.859649,NaN


Note: dropping NaN rows means that this code only checks for inter-annotator agreement if BOTH annotators have annotated the text. It does not account for cases where one annotator found metaphors in the text and another does not annotate anything.

In [14]:
compare_df = pd.merge(annotator1_df, annotator2_df, on=['text_id', 'text'], suffixes=('_1', '_2'))
# annotator columns are redundant now
compare_df = compare_df.drop(columns=['annotator_1', 'annotator_2'])
# drop any rows without annotations
compare_df = compare_df.dropna()

# compare_df.head()

In [15]:
# sanity check for how many rows are getting compared
len(compare_df)

25

In [32]:
for index, row in compare_df.iterrows():
    r=row['label_1']
    print(r)
    print(labels_to_list(r))
    agreement = compare_labels(row, 'label_1', 'label_2')
    print(agreement)

[{"start":322,"end":326,"text":"bill","labels":["METAPHOR"]},{"start":314,"end":317,"text":"fit","labels":["METAPHOR"]},{"start":38,"end":43,"text":"holds","labels":["METAPHOR"]},{"start":48,"end":55,"text":"highest","labels":["METAPHOR"]}]
[[322, 326], [314, 317], [38, 43], [48, 55]]
[1.0, nan]
[{"start":48,"end":58,"text":"shell game","labels":["METAPHOR"]},{"start":168,"end":173,"text":"crack","labels":["METAPHOR"]},{"start":212,"end":219,"text":"cracked","labels":["METAPHOR"]}]
[[48, 58], [168, 173], [212, 219]]
[1.0, nan]
[{"start":138,"end":144,"text":"thrive","labels":["METAPHOR"]},{"start":53,"end":62,"text":"substance","labels":["METAPHOR"]},{"start":125,"end":130,"text":"knows","labels":["METAPHOR"]},{"start":155,"end":160,"text":"knows","labels":["METAPHOR"]}]
[[138, 144], [53, 62], [125, 130], [155, 160]]
[1.0, nan]
[{"start":72,"end":76,"text":"he##","labels":["METAPHOR"]},{"start":294,"end":302,"text":"draining","labels":["METAPHOR"]},{"start":670,"end":685,"text":"limous

C:\Users\romha\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\metrics\_classification.py:407: UserWarning: A single label was found in 'y_true' and 'y_pred'. For the confusion matrix to have the correct shape, use the 'labels' parameter to pass all known labels.
  warnings.warn(
C:\Users\romha\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\metrics\_classification.py:758: RuntimeWarning: invalid value encountered in scalar divide
  k = np.sum(w_mat * confusion) / np.sum(w_mat * expected)
C:\Users\romha\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\metrics\_classification.py:407: UserWarning: A single label was found in 'y_true' and 'y_pred'. For the confusion matrix to have the correct shape, use the 'labels' parameter to pass all know

In [35]:
labels_to_list(r)

[[419, 432]]

In [16]:
textual_agreement = []
overlap_agreement = []

for index, row in compare_df.iterrows():
  agreement = compare_labels(row, 'label_1', 'label_2')
  textual_agreement.append(agreement[0])
  overlap_agreement.append(agreement[1])

compare_df['textual_agreement'] = textual_agreement
compare_df['overlap_agreement'] = overlap_agreement

C:\Users\romha\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\metrics\_classification.py:407: UserWarning: A single label was found in 'y_true' and 'y_pred'. For the confusion matrix to have the correct shape, use the 'labels' parameter to pass all known labels.
  warnings.warn(
C:\Users\romha\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\metrics\_classification.py:758: RuntimeWarning: invalid value encountered in scalar divide
  k = np.sum(w_mat * confusion) / np.sum(w_mat * expected)
C:\Users\romha\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\metrics\_classification.py:407: UserWarning: A single label was found in 'y_true' and 'y_pred'. For the confusion matrix to have the correct shape, use the 'labels' parameter to pass all know
























As per [this](https://datatab.net/tutorial/cohens-kappa) explanation of cohen's kappa, below is a guideline for how the result can be interpreted:
* \> 0.8 - almost perfect
* \> 0.6 - substantial
* \> 0.4 - moderate
* \> 0.2 - fair
* 0 - 0.2 - slight
* < 0 - poor

In [17]:
print("Average textual agreement: ", compare_df['textual_agreement'].mean())
print("Average overlap agreement: ", compare_df['overlap_agreement'].mean())

Average textual agreement:  0.7757065536440007
Average overlap agreement:  0.6375


In [36]:
labels_to_list('[{"start":322,"end":326,"text":"bill","labels":["METAPHOR"]},{"start":314,"end":317,"text":"fit","labels":["METAPHOR"]},{"start":38,"end":43,"text":"holds","labels":["METAPHOR"]},{"start":48,"end":55,"text":"highest","labels":["METAPHOR"]}]')

[[322, 326], [314, 317], [38, 43], [48, 55]]